# Importing .html files
### Analysis and calculations are in `madrid_housing_data_analysis.ipynb`.

In [3]:
#! unzip ~/madrid_housing_data_analysis/src/zipped.zip -d ~/madrid_housing_data_analysis/src/
# This command unzips the HTML files

In [4]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [5]:
import os
from os import listdir

# Create a list of all the file names in the '/src/' directory where .html files were saved
cwd = os.getcwd()
files = os.listdir(cwd + '/src/') 


In [6]:
counter = 0
limit = 10 # number of files to be read
for file in files: # Loops through all the files in the directory and creates a soup from each of them
    try: #so that it doenst raise an error if the file is not openable
        with open('src/'+ file ) as fp:
            print(file)
            print(counter)
            soup = BeautifulSoup(fp, 'html.parser')
            g_txt = soup.get_text()
            x = g_txt.split('var utag_data = ')[1].rsplit(';\n\t')
            try:
                latitude = float(g_txt.split("latitude: '")[1].rsplit("',")[0])
                longitude = float(g_txt.split("longitude: '")[1].rsplit("',")[0]) #latitude and longidue in float format
            except:
                latitude = 0
                longitude = 0
            try:
                distrito = g_txt.split('"header-map-list">Distrito ')[1].rsplit("</li><li class=")[0]
            except:
                distrito = "NaN"
            try:
                barrio = g_txt.split('"header-map-list">Barrio ')[1].rsplit("</li><li class=")[0]
            except: 
                barrio = "NaN"
            try:
                extra = g_txt.split('<span>Planta ')[1].rsplit("</p>")[0]
            except:
                extra = '000000'
            try:
                planta = int(extra.rsplit("ª</span>")[0])
            except:
                planta = 0
            try:
                garaje = extra.split('<span><span>')[1].rsplit("</span></span>")[0]
                if len(garaje)>16: #if it would contain too long of a sentence, it means it's optional            
                    garaje_coste = garaje.split('</span> ')[1].rsplit(" €")[0]
                    garaje_coste = float(garaje_coste.replace('.', ''))
                    garaje = 'optional'
                else:
                    garaje_coste = 0

            except: 
                garaje = 'None'
                garaje_coste = 0

            # check if flat is interior or exterior:
            try: 
                if 'exterior' in extra:
                    int_ext = 'exterior'
                elif 'interior' in extra:
                    int_ext = 'interior'
                elif ('exterior' in g_txt) & ('interior' in g_txt): 
                    int_ext = 'both'
                elif 'exterior' in g_txt: 
                    int_ext = 'exterior'
                elif 'interior' in g_txt:
                    int_ext = 'interior'
                else: int_ext = 'none'
            except: 
                int_ext = 'error'
            # check if flat has elevator or not
            try: 
                if 'con ascensor' in extra:
                    elevator = 'yes'
                elif 'sin ascensor' in extra:
                    elevator = 'no'
                else: elevator = 'unknown'
            except: 
                elevator = 'unknown'


            #planta = g_txt.
            json_object = json.loads(x[0])
            df_nested_list = pd.json_normalize(json_object)
            df_nested_list['latitude'] = latitude 
            df_nested_list['longitude'] = longitude
            df_nested_list['distrito'] = distrito # searches district
            df_nested_list['barrio'] = barrio # searches neighborhood
            df_nested_list['planta'] = planta # level
            df_nested_list['garaje'] = garaje # garage
            df_nested_list['garaje_coste'] = garaje_coste #cost of garage
            df_nested_list['int_ext'] = int_ext # interior o exterior
            df_nested_list['elevator'] = elevator # whether the house has elevator or not

            if counter == 0: # initialization of DataFrame
                col_headers = list(df_nested_list.columns.values)
                df = pd.DataFrame(columns = col_headers) 

            df = pd.concat([df, df_nested_list])
            counter +=1

            if counter == limit:
                break
    except:
        continue


view-source_https___www.idealista.com_inmueble_100001882_.html
0
view-source_https___www.idealista.com_inmueble_100002266_.html
1
view-source_https___www.idealista.com_inmueble_100003146_.html
2
view-source_https___www.idealista.com_inmueble_100004437_.html
3
view-source_https___www.idealista.com_inmueble_100005820_.html
4
view-source_https___www.idealista.com_inmueble_100006082_.html
5
view-source_https___www.idealista.com_inmueble_100008226_.html
6
view-source_https___www.idealista.com_inmueble_100012610_.html
7
view-source_https___www.idealista.com_inmueble_100013407_.html
8
view-source_https___www.idealista.com_inmueble_100013802_.html
9


In [7]:
x_text_1 = 'this one does have exterior and interior alike'
x_text_2 = 'this one does have exterior'
x_text_3 = 'this one has nothing'
x_list = [x_text_1, x_text_2, x_text_3]

for x in x_list: 
    if ('exterior' in x) & ('interior' in x):
        print('both')
    elif 'exterior' in x:
        print('exterior')
    elif 'interior' in x:
        print('interior')
    else:
        print('none')


both
exterior
none


In [8]:
useful_cols = [  # Selection of columns that will be saved to CSV and used during data analysis 
    'ad.id',         
    'ad.builtType',
    'ad.address.locationId', 
    'ad.energyCertification.type',
    'ad.energyCertification.suffix', 
    'ad.price',
    'ad.characteristics.roomNumber',
    'ad.characteristics.bathNumber',
    'ad.characteristics.hasLift',
    'ad.characteristics.constructedArea',
    'ad.condition.isNewDevelopment',
    'ad.condition.isNeedsRenovating',
    'ad.condition.isGoodCondition',
    'ad.owner.type',
    'ad.isSuitableForRecommended',
    'ad.numberRecommended',
    'longitude',
    'latitude',
    'distrito',
    'barrio',
    'planta',
    'garaje',
    'garaje_coste',
    'int_ext',
    'elevator'
]

In [9]:
df

,didomiVendorsConsent,page.section,page.subSection,page.name,page.language,page.category,page.subCategory,page.subSubCategory,page.template,page.position,...,post.time,latitude,longitude,distrito,barrio,planta,garaje,garaje_coste,int_ext,elevator
0,,portal,detail,home,es,detail,,,detail,,...,1676319198713,40.386156,-3.703812,Usera,Moscardó,5,None,0,exterior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676317475107,40.392456,-3.693902,Arganzuela,Delicias,1,optional,10000.0,interior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676303312932,40.405588,-3.706821,Arganzuela,Acacias,3,None,0,exterior,no
0,,portal,detail,home,es,detail,,,detail,,...,1676299803414,40.401151,-3.691816,Arganzuela,Palos de Moguer,4,None,0,interior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676302338619,40.399102,-3.683840,Arganzuela,Delicias,5,Garaje incluido,0,exterior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676321350189,40.369952,-3.689934,Usera,San Fermín,7,Garaje incluido,0,exterior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676323562606,0.000000,0.000000,NaN,NaN,0,None,0,none,unknown
0,,portal,detail,home,es,detail,,,detail,,...,1676316873256,40.392019,-3.692013,Arganzuela,Delicias,1,None,0,interior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676482498817,40.406269,-3.677909,Retiro,Niño Jesús,2,None,0,exterior,yes
0,,portal,detail,home,es,detail,,,detail,,...,1676319391227,40.387823,-3.715400,Usera,Moscardó,0,None,0,exterior,unknown


In [10]:
#conversion of datatypes
df['ad.price'] = df['ad.price'].astype('float')
df['ad.characteristics.roomNumber'] = df['ad.characteristics.roomNumber'].astype('float')
df['ad.characteristics.bathNumber'] = df['ad.characteristics.bathNumber'].astype('float')
df['ad.characteristics.hasLift'].fillna(0, inplace=True)
df['ad.characteristics.hasLift'] = df['ad.characteristics.hasLift'].astype('int')
df['ad.characteristics.constructedArea'] = df['ad.characteristics.constructedArea'].astype('float')
df['ad.condition.isNewDevelopment'] = df['ad.condition.isNewDevelopment'].astype('int')
df['ad.condition.isNeedsRenovating'] = df['ad.condition.isNeedsRenovating'].astype('int')
df['ad.condition.isGoodCondition'] = df['ad.condition.isGoodCondition'].astype('int')
df['garaje_coste'] = df['garaje_coste'].astype('float')

# adding new column: 
df['price_per_sqm'] = df['ad.price']/df['ad.characteristics.constructedArea']

In [12]:
# df.to_csv('madrid_housing_data.csv', encoding='utf-8') # Uncomment to export CSV